In [21]:
import pandas as pd
import tensorflow as tf 
import tensorflow_hub as hub
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [2]:
df = pd.read_csv("./data/song_lyrics_sampled_proc.csv")
print('Data rows number: ', len(df))
df.head()

Data rows number:  3919


,title,tag,artist,year,lyrics,id,lyrics_proc,lyrics_num_tks
0,Toothpick,pop,Biting Elbows,2012,Some folks got the patience of the angels\nNot...,1166787,folk got patience angel heart well yearns veng...,169
1,6 Feet Under,pop,Ana Johnsson,2004,You just left me 6 feet under ground I'm burni...,803057,left foot ground burning sight light foot buri...,100
2,The Poetaster Act 4. Scene 2,misc,Ben Jonson,1601,"A Room in Lupus's House.\n\nEnter Lupus, HISTR...",674438,room house enter lupus histrio lictors tuc sta...,170
3,Hes Gone,pop,Phil Lesh & Friends,2015,"Rat in a drain ditch, caught on a limb, you kn...",961823,rat drain ditch caught limb know better know l...,118
4,Ill Never Say,pop,Helen Ward,2015,"I'll never say ""never again"" again\nCause here...",1163619,never say never cause love head heel love neve...,35


### Load USE with TF

In [3]:
# Load the Universal Sentence Encoder module (transformer-based variant)
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use = hub.load(module_url)

lyrics_embeddings = df['lyrics_proc'].apply(lambda lyrics: use([lyrics]).numpy()[0])
lyrics_embeddings

2024-05-24 15:52:28.932831: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-05-24 15:52:33.133078: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


0       [-0.056033246, -0.054995343, 0.027326314, 0.01...
1       [-0.052749544, -0.04411805, 0.056027986, 0.040...
2       [-0.014596429, -0.057172682, -0.021360282, 0.0...
3       [-0.048244074, -0.05550254, 0.05564705, 0.0259...
4       [0.03838014, -0.02164263, -0.010910849, -0.042...
                              ...                        
3914    [-0.060502596, -0.059904378, 0.06110249, 0.026...
3915    [-0.056997176, -0.07835833, -0.031518035, 0.07...
3916    [-0.002709979, -0.05377683, 0.056938853, 0.053...
3917    [0.022438658, -0.06232634, 0.06601431, 0.06494...
3918    [-0.05009394, -0.035998333, 0.051654447, -0.00...
Name: lyrics_proc, Length: 3919, dtype: object

In [4]:
df['use_embeddings'] = lyrics_embeddings
df.head()

,title,tag,artist,year,lyrics,id,lyrics_proc,lyrics_num_tks,use_embeddings
0,Toothpick,pop,Biting Elbows,2012,Some folks got the patience of the angels\nNot...,1166787,folk got patience angel heart well yearns veng...,169,"[-0.056033246, -0.054995343, 0.027326314, 0.01..."
1,6 Feet Under,pop,Ana Johnsson,2004,You just left me 6 feet under ground I'm burni...,803057,left foot ground burning sight light foot buri...,100,"[-0.052749544, -0.04411805, 0.056027986, 0.040..."
2,The Poetaster Act 4. Scene 2,misc,Ben Jonson,1601,"A Room in Lupus's House.\n\nEnter Lupus, HISTR...",674438,room house enter lupus histrio lictors tuc sta...,170,"[-0.014596429, -0.057172682, -0.021360282, 0.0..."
3,Hes Gone,pop,Phil Lesh & Friends,2015,"Rat in a drain ditch, caught on a limb, you kn...",961823,rat drain ditch caught limb know better know l...,118,"[-0.048244074, -0.05550254, 0.05564705, 0.0259..."
4,Ill Never Say,pop,Helen Ward,2015,"I'll never say ""never again"" again\nCause here...",1163619,never say never cause love head heel love neve...,35,"[0.03838014, -0.02164263, -0.010910849, -0.042..."


In [5]:
def find_optimal_pca_dimensions(df, embeddings_column, dimensions, n_clusters=5):
    
    # Extract the embeddings from the DataFrame
    embeddings = np.stack(df[embeddings_column].values)
    
    # Apply Standard Scaler
    scaler = StandardScaler()
    scaled_embeddings = scaler.fit_transform(embeddings)
    
    results = []
    for dim in dimensions: 
        # Apply PCA
        pca = PCA(n_components=dim)
        reduced_embeddings = pca.fit_transform(scaled_embeddings)

        # Cluster the reduced embeddings
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(reduced_embeddings)

        # Compute the Silhouette score
        silhouette_avg = silhouette_score(reduced_embeddings, cluster_labels)

        # Calculate variance preserved
        variance_preserved = np.sum(pca.explained_variance_ratio_)

        # Append results
        results.append({
            'Dimensions': dim,
            'Silhouette Score': silhouette_avg,
            'Variance Preserved': variance_preserved
        })

    return pd.DataFrame(results)

dimensions = [2,3,5,10,15,20,35,50]

table = find_optimal_pca_dimensions(df,'use_embeddings',dimensions)
table

,Dimensions,Silhouette Score,Variance Preserved
0,2,0.395290,0.140073
1,3,0.376400,0.168047
2,5,0.282134,0.212485
3,10,0.183740,0.283918
4,15,0.166012,0.334330
5,20,0.147264,0.375500
6,35,0.116929,0.474418
7,50,0.075017,0.549799


In [16]:
def find_best_tradeoff(results_df):
    
    # Calculate the absolute difference between the normalized metrics
    results_df['Difference'] = np.abs(results_df['Silhouette Score'] - results_df['Variance Preserved'])
    
    # Find the dimension with the minimum difference
    best_dimension = results_df.loc[results_df['Difference'].idxmin(), 'Dimensions']
    
    return best_dimension, results_df

best_dim, _ = find_best_tradeoff(table)
print("The best dimension is: ", best_dim)

# Extract the embeddings from the DataFrame
embeddings = np.stack(df['use_embeddings'].values)

# Apply Standard Scaler
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(embeddings)

# Apply PCA
pca = PCA(n_components=best_dim)
reduced_embeddings = pca.fit_transform(scaled_embeddings)

# Add reduced embedding as new column
df['reduced_embeddings'] = list(reduced_embeddings)
df

The best dimension is:  5


,title,tag,artist,year,lyrics,id,lyrics_proc,lyrics_num_tks,use_embeddings,reduced_embeddings
0,Toothpick,pop,Biting Elbows,2012,Some folks got the patience of the angels\nNot...,1166787,folk got patience angel heart well yearns veng...,169,"[-0.056033246, -0.054995343, 0.027326314, 0.01...","[0.8184663, 8.934743, 0.81324434, -2.563881, -..."
1,6 Feet Under,pop,Ana Johnsson,2004,You just left me 6 feet under ground I'm burni...,803057,left foot ground burning sight light foot buri...,100,"[-0.052749544, -0.04411805, 0.056027986, 0.040...","[-2.9089687, 3.2278845, -1.5783634, -5.9544272..."
2,The Poetaster Act 4. Scene 2,misc,Ben Jonson,1601,"A Room in Lupus's House.\n\nEnter Lupus, HISTR...",674438,room house enter lupus histrio lictors tuc sta...,170,"[-0.014596429, -0.057172682, -0.021360282, 0.0...","[2.7934368, 6.4212475, 4.939221, -2.6193573, 0..."
3,Hes Gone,pop,Phil Lesh & Friends,2015,"Rat in a drain ditch, caught on a limb, you kn...",961823,rat drain ditch caught limb know better know l...,118,"[-0.048244074, -0.05550254, 0.05564705, 0.0259...","[0.7794925, 8.609817, 0.43516323, 1.4209435, 0..."
4,Ill Never Say,pop,Helen Ward,2015,"I'll never say ""never again"" again\nCause here...",1163619,never say never cause love head heel love neve...,35,"[0.03838014, -0.02164263, -0.010910849, -0.042...","[-4.853679, -0.82479984, -5.673372, 2.298163, ..."
...,...,...,...,...,...,...,...,...,...,...
3914,I Think About You,pop,David Devant & His Spirit Wife,1997,Now get this clear\nI'm being sincere\nYour bi...,935693,get clear sincere biggest fan also ran someone...,103,"[-0.060502596, -0.059904378, 0.06110249, 0.026...","[-5.32884, 1.2038362, -5.728632, -0.44328254, ..."
3915,About Face,rock,Have Heart,2006,Fucked it up again but I'm not looking the oth...,1244191,fucked looking way see error know wrong going ...,61,"[-0.056997176, -0.07835833, -0.031518035, 0.07...","[-3.2777505, -5.717128, 4.879248, 1.4938534, -..."
3916,Let Em Know,pop,Grade 8,2003,Spit shit right into my ear kid\nI don't wanna...,924470,spit shit right ear kid wan hear look alive fe...,167,"[-0.002709979, -0.05377683, 0.056938853, 0.053...","[-0.40754858, 9.754164, 1.0369285, -1.8669525,..."
3917,Wings Of Desire,rock,Wishbone Ash,1991,I've seen you in black and white\nLike a silen...,1053952,seen black white like silent movie colour came...,70,"[0.022438658, -0.06232634, 0.06601431, 0.06494...","[-5.5200343, -2.4140143, -2.6857204, 0.6594804..."


In [19]:
def plot_elbow_curve(df, embeddings_column='reduced_embeddings', cluster_range=(2, 50)):
    # Extract the embeddings from the DataFrame
    reduced_embeddings = np.stack(df[embeddings_column].values)

    # Initialize variables to store inertia values
    cluster_range_values = range(cluster_range[0], cluster_range[1] + 1)
    inertia_values = []

    # Compute inertia for each number of clusters
    for n_clusters in cluster_range_values:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        kmeans.fit(reduced_embeddings)
        inertia_values.append(kmeans.inertia_)

    # Calculate the best number of clusters using the elbow method
    deltas = np.diff(inertia_values, 2)
    elbow_index = np.argmin(deltas) + 1  # Add 1 because of the diff operation
    best_clusters = cluster_range_values[8]

    # Plot the elbow curve using Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=list(cluster_range_values)[::1],
        y=inertia_values,
        mode='lines+markers',
        name='Inertia'
    ))

    fig.add_shape(
        type="line",
        x0=best_clusters,
        y0=min(inertia_values),
        x1=best_clusters,
        y1=max(inertia_values),
        line=dict(
            color="red",
            width=2,
            dash="dashdot",
        ),
    )

    fig.update_layout(
        title='Elbow Curve for KMeans Clustering',
        xaxis_title='Number of Clusters',
        yaxis_title='Inertia',
        xaxis=dict(tickmode='linear', dtick=5),
        template='plotly_white'
    )

    fig.show()
    
plot_elbow_curve(df)

In [22]:
def perform_clustering_and_visualize(df, embeddings_column='reduced_embeddings', n_clusters=10):
    # Extract the embeddings from the DataFrame
    reduced_embeddings = np.stack(df[embeddings_column].values)

    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(reduced_embeddings)

    # Apply t-SNE for visualization
    tsne = TSNE(n_components=2, random_state=42)
    tsne_embeddings = tsne.fit_transform(reduced_embeddings)

    # Create a DataFrame for visualization
    tsne_df = pd.DataFrame(tsne_embeddings, columns=['x', 'y'])
    tsne_df['cluster'] = cluster_labels

    # Visualize using Plotly
    fig = px.scatter(
        tsne_df, x='x', y='y', color='cluster', 
        title='t-SNE Visualization of Clusters',
        labels={'x': 't-SNE Dimension 1', 'y': 't-SNE Dimension 2'}
    )
    fig.show()
    return cluster_labels

cluster_labels = perform_clustering_and_visualize(df)
# Add cluster labels to the lyrics embeddings
df['cluster'] = cluster_labels

In [47]:
def top_n_similar_embeddings(df, target_index, n=10):
    
    # Extract the target embedding and its cluster
    target_embedding = df.at[target_index, 'reduced_embeddings']
    target_cluster = df.at[target_index, 'cluster']
    
    # Filter the DataFrame to get only the embeddings in the same cluster
    cluster_df = df[df['cluster'] == target_cluster]
    cluster_embeddings = np.stack(cluster_df['reduced_embeddings'].values)

    # Calculate cosine similarities
    similarities = cosine_similarity([target_embedding], cluster_embeddings)[0]

    # Get the top n similar embeddings
    top_n_indices = np.argsort(similarities)[-n-1:][::-1]
    top_n_similar_ids = cluster_df.iloc[top_n_indices]['id'].values
    top_n_similarities = similarities[top_n_indices]

    return pd.DataFrame({
        'id': top_n_similar_ids, 
        'similarity': top_n_similarities
    })

# Show top 10 similar song to the first song (the first row is the song itself)
top_n_similar_embeddings(df,target_index=0,n=10)

,id,similarity
0,1166787,1.000000
1,428247,0.987543
2,343812,0.987366
3,924470,0.983680
4,815153,0.972437
5,357575,0.969552
6,539543,0.963986
7,1090672,0.962836
8,169858,0.959384
9,38248,0.952045
